In [22]:
from openai import OpenAI
from dotenv import load_dotenv

client = OpenAI()

#client = OpenAI(api_key="...") #add api key manually here


In [23]:
import os
import base64
import requests
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [24]:
# Function to base-64 encode images
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

class ChatBot:
  
    def __init__(self, 
                 api_key=os.getenv('OPENAI_API_KEY'), 
                 model='gpt-4o', 
                 system='You are a helpful assistant.') -> None:
        self._api_key = api_key
        self._model = model
        self._system = system

        self._messages = [{"role": "system", "content": self._system}]
        self._max_tokens = 2000
        self._client = OpenAI(api_key=self._api_key)

    @property
    def _headers(self):
        return {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self._api_key}"
        }
    
    @property
    def _payload(self):
        return {
            "model": self._model,
            "messages": self.messages,
            "max_tokens": self._max_tokens
        }

    @property
    def messages(self):
       return self._messages
    
    def clear_messages(self):
       self._messages = []

    def _add_message(self, message):
       self._messages.append(message)

    def _ask_text(self, text):
        self._add_message(
           {"role": "user", "content": text},
        )                

        response = client.chat.completions.create(
            model=self._model,
            messages=self.messages,
            max_tokens=self._max_tokens
        )

        
        self._add_message({k: v for (k, v) in response.choices[0].message if k in ['role', 'content']})  

        return response.choices[0].message.content
    
    def _ask_image(self, text, image_path):
        image_b64 = encode_image(image_path)
        
        self.messages.append({
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": text
                },
                {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encode_image(image_path)}"
                }
                }
            ]
        })

        # get response
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=self._headers, json=self._payload)

        if response.status_code != 200:
            return f"Error {response.status_code}: {response.text}"

        # add to chat history
        message = response.json()['choices'][0]['message']
        self._add_message(message)
        return message['content']
    
    def chat(self, text, image_path=None):
       if not image_path:
          return self._ask_text(text)
       else:
          return self._ask_image(text, image_path)

In [25]:
bot = ChatBot()

In [35]:
bot = ChatBot()
print(bot.chat("""
explain me this picture. 
""".strip(),
image_path='Poster Presentation_page-0001.jpg'
))

This image is a research poster about using Conditional Generative Adversarial Networks (cGANs) for classifying brain tumor MRI images. Here's a breakdown:

### Title & Author
- The study is titled "Data Augmentation with Conditional Generative Adversarial Networks (cGANs) for Deep Learning-based Classification of Brain Tumor Magnetic Resonance Images."
- Authored by Mahnoor Mahnoor from Turku PET Centre, University of Turku.

### Background
- The study aims to validate a deep learning model for generating synthetic MRI images of brain tumors.
- It addresses issues like data scarcity and model instability often faced in neural networks.

### Methods
- Uses a cGAN framework:
  - **Generator**: Creates synthetic images from noise.
  - **Discriminator**: Differentiates between real and fake images, improving the model.
- Architecture: U-Net architecture is used in the cGAN for image processing.

### Results
- Synthetic images significantly improved CNN's performance in classifying brain t

In [36]:
bot.chat("give me suggestion how can i imrpove this picture. also tell me what tools should i use?")

'Improving your research poster involves enhancing clarity, visual appeal, and communication of information. Here are some suggestions:\n\n### Content Enhancements\n1. **Simplify Text:**\n   - Use bullet points for easy reading.\n   - Avoid jargon and keep explanations concise.\n\n2. **Highlight Key Points:**\n   - Use bold text or color to highlight important findings and conclusions.\n\n3. **Logical Flow:**\n   - Ensure sections are ordered logically, guiding the viewer through the research process.\n\n### Visual Improvements\n1. **Consistent Color Scheme:**\n   - Use a cohesive color palette to improve aesthetics and readability.\n\n2. **High-Quality Images:**\n   - Ensure all images and graphs are high-resolution for clarity.\n\n3. **Balanced Layout:**\n   - Utilize whitespace effectively to prevent the poster from looking cluttered.\n\n### Graphical Enhancements\n1. **Graph Labels:**\n   - Ensure all axes and legends in graphs are clearly labeled.\n\n2. **Visual Hierarchy:**\n   -

In [37]:
print(bot.messages[-1]['content'])

Improving your research poster involves enhancing clarity, visual appeal, and communication of information. Here are some suggestions:

### Content Enhancements
1. **Simplify Text:**
   - Use bullet points for easy reading.
   - Avoid jargon and keep explanations concise.

2. **Highlight Key Points:**
   - Use bold text or color to highlight important findings and conclusions.

3. **Logical Flow:**
   - Ensure sections are ordered logically, guiding the viewer through the research process.

### Visual Improvements
1. **Consistent Color Scheme:**
   - Use a cohesive color palette to improve aesthetics and readability.

2. **High-Quality Images:**
   - Ensure all images and graphs are high-resolution for clarity.

3. **Balanced Layout:**
   - Utilize whitespace effectively to prevent the poster from looking cluttered.

### Graphical Enhancements
1. **Graph Labels:**
   - Ensure all axes and legends in graphs are clearly labeled.

2. **Visual Hierarchy:**
   - Size and place key sections 

In [38]:
print(bot.chat("help me rewrite the text in it."))

Certainly! Here's a more concise and reader-friendly version of the text:

---

### Title
**Enhancing Brain Tumor MRI Classification with cGAN-Generated Data**

### Background
This study explores using cGANs to generate synthetic brain tumor MRI images, boosting the performance of deep learning models. It addresses issues like data scarcity and model instability.

### Methods
- **cGAN Framework:**
  - **Generator:** Produces synthetic images from noise.
  - **Discriminator:** Differentiates real from fake images to refine model accuracy.
- **Architecture:** Utilizes U-Net for enhanced image processing.

### Results
- Synthetic images improved CNN classification accuracy.
- Combining real and synthetic data enhanced diagnostic precision.

### Discussion
Synthetic MRI images resolve issues such as limited datasets and privacy concerns, making machine learning more viable in clinical settings.

### Conclusion
Incorporating synthetic data with CNNs significantly boosts brain tumor classifi